In [ ]:
import pandas as pd
import sqlite3

## SQL Hepler Functions

In [ ]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

## Execute Query

In [ ]:
# Step 1: Read the csv file into a dataframe
claims_input_df = pd.read_csv('https://raw.githubusercontent.com/melocotonen/data-intquesDS/main/sql/data/claims.csv')
policies_input_df = pd.read_csv('https://raw.githubusercontent.com/melocotonen/data-intquesDS/main/sql/data/policies.csv')
users_input_df = pd.read_csv('https://raw.githubusercontent.com/melocotonen/data-intquesDS/main/sql/data/users.csv')
 
# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(claims_input_df,
            table_name='claims',
            db_name='evalua.db')
pd_to_sqlDB(policies_input_df,
            table_name='policies',
            db_name='evalua.db')
pd_to_sqlDB(users_input_df,
            table_name='users',
            db_name='evalua.db')


# Prueba codificación

1) AAABBB. Imprime números del 1 al 100

* Si es múltiplo de 3, imprime "AAA"
* Si es múltiplo de 5, imprime "BBB"
* Si es múltiplo de 5, imprime "BBB"
* Si es de ambos — “AAA BBB"
* Sino imprime el número

Ejemplo de salida: 1, 2, AAA, 4, BBB, AAA, 7, 8, AAA, BBB, 11, AAA, 13, 14, AAA BBB, 16, 17, AAA, 19, BBB, AAA, 22, 23, AAA, BBB, 26, AAA, 28, 29, AAA BBB, 31, 32, AAA, 34, BBB, AAA, ...

2) Factorial. Escribe el factorial de un número


factorial(4) = 4! = 1 * 2 * 3 * 4 = 24

3) Media. Calcula la media de una lista

avg([4, 36, 45, 50, 75]) = 42

4) Borras duplicados. Elimina duplicados de una lista La lista no está ordenada y el orden de la lista final debe de ser la misma

[1, 2, 3, 2] ⇒ [1, 2, 3]

[1, 2, 2, 1, 5, 3, 2, 1, 4] ⇒ [1, 3, 2, 5, 4]

5) Contar. Cuantas veces se repite el elemento de una lista

[1, 3, 2, 1, 5, 3, 5, 1, 4] ⇒

* 1: 3 veces
* 3: 2 veces
* 2: 1 vez
* 5: 2 veces
* 4: 1 vez

# Prueba SQL



 ### Número de pólizas activas

In [ ]:
sql_query_string = """
    SELECT *
    from claims
"""
 
result_df = sql_query_to_pd(sql_query_string, db_name='evalua.db')
result_df

,polizas_activas
0,491


### Listado de todas las campañías activas. Una compañia está activa si al menos hay una poliza activa (company_id)

In [ ]:
sql_query_string = """
    SELECT *
    from claims
"""
 
result_df = sql_query_to_pd(sql_query_string, db_name='evalua.db')
result_df

,id,policy_id,claim_date,claim_type
0,d6f0356d-721f-49c2-b5af-2aa3a4c0884f,1564c62a-777e-4c7e-9161-e960d311f286,14/04/2022,Mixto
1,28342a1f-2476-4a9f-80b5-a78df9e2b670,49648683-77fc-49d4-a156-485706f9aa2d,06/03/2022,Mixto
2,c7d6788f-975b-451f-b822-03c06aeb0180,67fed84d-74ee-4bad-aa33-918f911322a1,02/10/2022,Reparable
3,21604b81-19ed-45fa-824c-32452a0166c6,45d02c04-33ee-4b47-a026-fd214660dc40,08/08/2022,Mixto
4,670b3fd5-c0f6-4a7c-a535-488b98219c54,3592f667-119b-4295-a1c4-f0c9cf685aa8,24/02/2022,Mixto
...,...,...,...,...
995,9fdceea4-be81-4ffc-bd48-6c8fbab20730,0022b3ba-4fd6-4c00-b764-154c3401c8b5,17/02/2022,Reparable
996,1ff51e5d-4245-4447-9dec-d0d45dcff189,12d70555-5cc3-4d90-b892-a0868ee680cd,01/02/2022,Reparable
997,46dd7915-8340-40ef-9894-284a4cdf0f86,52b6d229-1706-4cbf-8131-f2febea67a1d,03/08/2022,Mixto
998,f8c97bf1-146c-4086-ad0a-fb0bd07bfb45,e774004d-9ff3-4a82-9588-b743a7569207,15/08/2022,Reparable


### Número de compañías activas

In [ ]:
sql_query_string = """
    SELECT *
    from claims
"""
 
result_df = sql_query_to_pd(sql_query_string, db_name='evalua.db')
result_df

,id,policy_id,claim_date,claim_type
0,d6f0356d-721f-49c2-b5af-2aa3a4c0884f,1564c62a-777e-4c7e-9161-e960d311f286,14/04/2022,Mixto
1,28342a1f-2476-4a9f-80b5-a78df9e2b670,49648683-77fc-49d4-a156-485706f9aa2d,06/03/2022,Mixto
2,c7d6788f-975b-451f-b822-03c06aeb0180,67fed84d-74ee-4bad-aa33-918f911322a1,02/10/2022,Reparable
3,21604b81-19ed-45fa-824c-32452a0166c6,45d02c04-33ee-4b47-a026-fd214660dc40,08/08/2022,Mixto
4,670b3fd5-c0f6-4a7c-a535-488b98219c54,3592f667-119b-4295-a1c4-f0c9cf685aa8,24/02/2022,Mixto
...,...,...,...,...
995,9fdceea4-be81-4ffc-bd48-6c8fbab20730,0022b3ba-4fd6-4c00-b764-154c3401c8b5,17/02/2022,Reparable
996,1ff51e5d-4245-4447-9dec-d0d45dcff189,12d70555-5cc3-4d90-b892-a0868ee680cd,01/02/2022,Reparable
997,46dd7915-8340-40ef-9894-284a4cdf0f86,52b6d229-1706-4cbf-8131-f2febea67a1d,03/08/2022,Mixto
998,f8c97bf1-146c-4086-ad0a-fb0bd07bfb45,e774004d-9ff3-4a82-9588-b743a7569207,15/08/2022,Reparable


### Numero de siniestros por cada poliza desglosado por tipo de siniestro



In [ ]:
sql_query_string = """
    SELECT *
    from claims
"""
 
result_df = sql_query_to_pd(sql_query_string, db_name='evalua.db')
result_df

,id,policy_id,claim_date,claim_type
0,d6f0356d-721f-49c2-b5af-2aa3a4c0884f,1564c62a-777e-4c7e-9161-e960d311f286,14/04/2022,Mixto
1,28342a1f-2476-4a9f-80b5-a78df9e2b670,49648683-77fc-49d4-a156-485706f9aa2d,06/03/2022,Mixto
2,c7d6788f-975b-451f-b822-03c06aeb0180,67fed84d-74ee-4bad-aa33-918f911322a1,02/10/2022,Reparable
3,21604b81-19ed-45fa-824c-32452a0166c6,45d02c04-33ee-4b47-a026-fd214660dc40,08/08/2022,Mixto
4,670b3fd5-c0f6-4a7c-a535-488b98219c54,3592f667-119b-4295-a1c4-f0c9cf685aa8,24/02/2022,Mixto
...,...,...,...,...
995,9fdceea4-be81-4ffc-bd48-6c8fbab20730,0022b3ba-4fd6-4c00-b764-154c3401c8b5,17/02/2022,Reparable
996,1ff51e5d-4245-4447-9dec-d0d45dcff189,12d70555-5cc3-4d90-b892-a0868ee680cd,01/02/2022,Reparable
997,46dd7915-8340-40ef-9894-284a4cdf0f86,52b6d229-1706-4cbf-8131-f2febea67a1d,03/08/2022,Mixto
998,f8c97bf1-146c-4086-ad0a-fb0bd07bfb45,e774004d-9ff3-4a82-9588-b743a7569207,15/08/2022,Reparable


# Prueba DS

La prueba está dividida en varios ejercicios rápidos que nos demostrarán tu manejo con pandas, datetime y numpy. Además en algunos de estos ejercicios deberás realizar visualizaciones que dejamos a tu elección el módulo a usar (plotly, matplot,seaborn...)

## Encuestas
- Dataset: encuestas_DATA
- Ejercicio: De la siguiente lista de encuestas, necesitamos conocer la diferencia entre la máxima puntuación total obtenida y la menor. La puntuación total es la suma de todas las puntuaciones.

In [ ]:
url = 'https://raw.githubusercontent.com/melocotonen/data-intquesDS/main/data_science/data/emails_DATA.csv'
encuestas_df = pd.read_csv(url)
encuestas_df

,id,nombre,email
0,6c10df0d-cd24-41c3-aee5-e4e40dd0c832,Lauraine,lminillo0@sciencedaily.com
1,e03ce4d0-5b2f-43ff-8b6d-a494ecf76da3,Clemente,ctulloch1@ucsd.edu
2,5ebc499f-1b52-47a0-99bd-11b3db2dbe58,Berte,bpury2@nationalgeographic.com
3,7990517a-b3f9-4f90-8151-002de0651c26,Lorita,lstable3@latimes.com
4,abac6dda-98e8-4123-b627-cafb7037cb43,Edy,ekeyzor4@pcworld.com
...,...,...,...
995,531fcbc4-86b7-485f-ab11-e226ba78e899,Ximenes,xlorkingrn@myspace.com
996,9de48add-27a4-4c3e-a3e4-83f32ff9cd8e,Joellyn,jmingusro@google.co.uk
997,6c8236b2-6796-40de-adc9-6f13045fc845,Constanta,cfawdreyrp@senate.gov
998,d8d1f513-2118-480d-bb80-e242f60015fc,Thebault,tdugallrq@technorati.com


## 2) Facturación
- Dataset: facturas_DATA
- Ejercicio: Devuelve la lista de encargos que tengan facturas de tipo 2 y 3 para un mismo encargo. Además, indica la suma de importes por encargo

In [ ]:
url = 'https://raw.githubusercontent.com/melocotonen/data-intquesDS/main/data_science/data/facturas_DATA.csv'
facturacion_df = pd.read_csv(url)
facturacion_df

,id,estado,encargo,tipo,importe
0,e37fb24c-1972-4b83-bcfa-9ce01e3a2c0f,4,368,2,923.05
1,954b698a-9df2-41a0-b788-6b53105b543c,3,196,3,244.35
2,e6aaa2fe-bbab-43e6-b2d1-4df3386e9ebd,3,176,3,746.70
3,799c8398-6136-4dd6-bdf4-a14d8576d1c4,2,324,3,929.73
4,6584de31-4b2e-4dcc-8cf0-bd18034d40bb,4,23,2,757.56
...,...,...,...,...,...
995,d76cf56e-1094-41d8-8f52-7756fabc2b1f,4,665,2,639.50
996,55ce508a-a6cb-430a-8fdc-bc5a67970580,1,242,1,964.98
997,9bbdd78d-a162-4370-8fd6-b87c2bf84d52,3,535,3,759.15
998,a12c754a-f2c0-4db3-9075-6f00dd9fbcf1,3,302,3,375.53


## 3) Emails duplicados
- Dataset: emails_DATA
- Ejercicio: Devuelve una lista de emails que estén duplicados y muestra de forma visual el número de emails duplicados y únicos. Puedes usar matplotlib, pyplot, seaborn o lo que prefieras.

In [ ]:
url = 'https://raw.githubusercontent.com/melocotonen/data-intquesDS/main/data_science/data/encuestas_DATA.csv'
encuestas_df = pd.read_csv(url)
encuestas_df

,id,first_name,score,score_2,score_3
0,70fb62c5-fd8a-4ba5-a3f0-27e80d91ae6a,Cindie,6,10,2
1,70fb62c5-fd8a-4ba5-a3f0-27e80d91ae6a,Cindie,6,10,2
2,751d423f-22d0-4434-95f0-1991b5a7e288,Edy,1,4,10
3,6f06a411-82ab-4893-833e-5b2bd0e95070,Buiron,2,8,6
4,32149065-7767-46ae-ab5e-8f9642a501fa,Any,8,4,4
...,...,...,...,...,...
996,07553588-ccd9-4673-9f55-f045ec3a0b21,Isaak,7,7,9
997,ce360be8-6ae3-481d-8e6f-5c1a484c2100,Annabelle,4,5,4
998,4d797068-f526-4457-a432-b10de70418eb,Kalle,8,9,3
999,03a40ae3-fb4c-4385-8186-5ef237a44e84,Hamil,9,5,2


## 4) Visualizaciones

Utilizando cualquiera de los datasets que se aportan para realizar visualizaciones que puedan aportar valor o que añadirías en un EDA.